## Working with simple databases using pandas

In [51]:
import sqlite3

### Connect to the database in memory

In [52]:
# dbconn = sqlite3.connect(':memory:')  # for a temporary database

In [53]:
dbconn = sqlite3.connect('mydw') # permanent database

In [54]:
print()

### Create a cursor to use to execute SQL statements.

In [55]:
cursor = dbconn.cursor()

### Use the cursor to create a sample databse and put that into a pandas dataframe

In [56]:
cursor.execute('''CREATE TABLE IF NOT EXISTS books(id INTEGER PRIMARY KEY, 
                   title TEXT, author TEXT, price TEXT, year TEXT)
''')
dbconn.commit()

In [57]:
cursor.execute('''INSERT INTO books values (1, 'Pro PowerShell', 'Bryan Cafferky', 35.00, 2015)
''')
cursor.execute('''INSERT INTO books values (2, "'Hithiker's Guide to the Galaxy", 'Douglas Adams', 12.00, 199)
''')
dbconn.commit()

In [58]:

import pandas as pd

# convert query results to a dataframe
dfbook = pd.read_sql_query("SELECT * FROM books", dbconn)
dfbook.head()

,id,title,author,price,year
0,1,Pro PowerShell,Bryan Cafferky,35.0,2015
1,2,'Hithiker's Guide to the Galaxy,Douglas Adams,12.0,199


In [59]:
type(dfbook)

pandas.core.frame.DataFrame

### Dropping the table

In [60]:
cursor = dbconn.cursor()
cursor.execute('''DROP TABLE books''')
dbconn.commit()

### Close the connection

In [61]:
cursor.close()
dbconn.close()

## Using a sample databse from SQLite

In [62]:
conn = sqlite3.connect("chinook.db")
cur = conn.cursor()

### Check the tables using databse catalog

In [63]:
pd.read_sql_query("select tbl_name from sqlite_master where type = 'table';", conn)

,tbl_name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


In [64]:
albums = cur.execute("""select * from albums order by title limit 3;""").fetchall()
print(albums)

[(156, '...And Justice For All', 50), (257, '20th Century Masters - The Millennium Collection: The Best of Scorpions', 179), (296, 'A Copland Celebration, Vol. I', 230)]


In [65]:
cur.execute("""select * from albums order by title limit 3;""").fetchone()

(156, '...And Justice For All', 50)

In [66]:
albumdata = cur.execute("""select * from albums order by title limit 3;""").fetchmany(5)

In [67]:
# Convert the list returned to a dataframe..

from pandas import DataFrame

DataFrame(albumdata)

,0,1,2
0,156,...And Justice For All,50
1,257,20th Century Masters - The Millennium Collecti...,179
2,296,"A Copland Celebration, Vol. I",230


In [68]:
import sqlite3
import pandas as pd

# return query results as a dataframe

dfalbum = pd.read_sql_query("SELECT * FROM albums", conn)
dfalbum.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


### Changing the default dataframe index

In [69]:
qry = '''
select substr(a.name, 1, 10) as Artist,
       substr(title,1, 15) as Album,
       t.name as Song
from albums r
join artists a
on (r.artistid = a.artistid)
join tracks t 
on (r.albumid = t.albumid)
order by a.name, title
limit 15;
'''

pd.read_sql_query(qry, conn, index_col = 'Artist')  # index_col replaces the default dataframe index

,Album,Song
Artist,,
AC/DC,For Those About,For Those About To Rock (We Salute You)
AC/DC,For Those About,Put The Finger On You
AC/DC,For Those About,Let's Get It Up
AC/DC,For Those About,Inject The Venom
AC/DC,For Those About,Snowballed
AC/DC,For Those About,Evil Walks
AC/DC,For Those About,C.O.D.
AC/DC,For Those About,Breaking The Rules
AC/DC,For Those About,Night Of The Long Knives


### Creating a new table from a SQL query 

In [70]:
# Note the create table statment before the select query...

qry = '''
create table if not exists top15 as
select substr(a.name, 1, 10) as Artist,
       substr(title,1, 15) as Album,
       t.name as Song
from albums r
join artists a
on (r.artistid = a.artistid)
join tracks t 
on (r.albumid = t.albumid)
order by a.name, title
limit 15;
'''

albums = cur.execute(qry)

In [71]:
pd.read_sql_query('select * from top15', conn)

,Artist,Album,Song
0,AC/DC,For Those About,For Those About To Rock (We Salute You)
1,AC/DC,For Those About,Put The Finger On You
2,AC/DC,For Those About,Let's Get It Up
3,AC/DC,For Those About,Inject The Venom
4,AC/DC,For Those About,Snowballed
5,AC/DC,For Those About,Evil Walks
6,AC/DC,For Those About,C.O.D.
7,AC/DC,For Those About,Breaking The Rules
8,AC/DC,For Those About,Night Of The Long Knives
9,AC/DC,For Those About,Spellbound


### Creating a new table from a dataframe

In [72]:
best_acdcDF = pd.read_sql_query('select * from top15 where artist = "AC/DC"', conn)

In [73]:
best_acdcDF.to_sql('best_acdc', con=conn, if_exists='append')

15

In [74]:
 pd.read_sql_query('select * from best_acdc', conn)

,index,Artist,Album,Song
0,0,AC/DC,For Those About,For Those About To Rock (We Salute You)
1,1,AC/DC,For Those About,Put The Finger On You
2,2,AC/DC,For Those About,Let's Get It Up
3,3,AC/DC,For Those About,Inject The Venom
4,4,AC/DC,For Those About,Snowballed
...,...,...,...,...
85,10,AC/DC,Let There Be Ro,Go Down
86,11,AC/DC,Let There Be Ro,Dog Eat Dog
87,12,AC/DC,Let There Be Ro,Let There Be Rock
88,13,AC/DC,Let There Be Ro,Bad Boy Boogie


### Closing the connection. 

In [75]:
cur.close()
conn.close()